# Домашняя работа №5. Метод опорных векторов

Цель работы: Использовать метод опорных векторов для решения задачи регрессии.

### Работа с набором данных для задачи регрессии (целевая переменная - непрерывное значение)

## Задание №1

#### Предварительная подготовка данных

Подготовить данные (если не выполнялось ранее, иначе использовать подготовленные данные):

* Заполнить пропуски в данных подходящими значениями (например, 0 для числовых значений, other для категориальных значений).
* Проверить датасет на дубликаты. Удалить при наличии.
* Закодировать категориальные значения. Например, создать новые булевые признаки (OneHotEncoder), или назначить число для каждого значения (LabelEncoder).
* Удалить столбцы с уникальными значениями, при наличии.
* К числовым признакам применить масштабирование со средним нулевым значением.
* Разделить на тренировочную и обучающую выборки.

## Задание №2

#### Обучение модели
* Обучить модель с помощь класса SVR в Sklearn с параметрами по умолчанию.
* Вычислить метрики r2, mse, mae на тренировочной и тестовой выборках.
* Сделать вывод о качестве обученной модели.

## Задание №3

#### Настройка гиперпараметров модели
* С помощью инструмента Optuna подобрать оптимальные гиперпараметры (ядро: Полиномиальное, Радиальное; С: от 1 до 100, гамма: 0.01 до 100)
* С помощью найденных лучших параметров обучить модель.
* Вычислить метрики полученной модели (r2, mse, mae) на тренировочной и тестовой выборках.

# Решение домашней работы №5

In [5]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

## Задание №1

Забрала тест и трэйн из домашки по линейной регрессии, вставив в тот коллаб следующий код:

In [ ]:
# X_train_df = pd.DataFrame(X_train)
# X_test_df = pd.DataFrame(X_test)
# y_train_df = pd.DataFrame(y_train)
# y_test_df = pd.DataFrame(y_test)
# X_train_df.to_csv('X_train.csv', index=False)
# X_test_df.to_csv('X_test.csv', index=False)
# y_train_df.to_csv('y_train.csv', index=False)
# y_test_df.to_csv('y_test.csv', index=False)

Прочитаем файлы:

In [6]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv').values.ravel()

## Задание №2

Обучение:

In [7]:
model = SVR()
model.fit(X_train, y_train)

SVR()

Прогноз:

In [8]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

Метрики:

In [9]:
r2_train = r2_score(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

r2_test = r2_score(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f'Метрики на тренировочном наборе:\n'
      f'r2: {r2_train:.4f}\n'
      f'mse: {mse_train:.4f}\n'
      f'mae: {mae_train:.4f}')

print(f'\nМетрики на тестовом наборе:\n'
      f'r2: {r2_test:.4f}\n'
      f'mse: {mse_test:.4f}\n'
      f'mae: {mae_test:.4f}')

Метрики на тренировочном наборе:
r2: 0.9972
mse: 0.0028
mae: 0.0441

Метрики на тестовом наборе:
r2: 0.9970
mse: 0.0031
mae: 0.0444


Вывод:
И на тестовой, и на обучающей выборках метрики говорят о хорошем обучении модели, так как R^2 указывает на высокую степень соответствия данных модели, а MSE и MAE показывают низкую степень ошибки

## Задание №3

In [11]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [12]:
import optuna
from sklearn.model_selection import cross_val_score

Определение целевой функции, которую Optuna будет минимизировать для поиска наилучших гиперпараметров модели

In [13]:
def objective(trial):
    kernel = trial.suggest_categorical("kernel", ["poly", "rbf"])
    C = trial.suggest_float("C", 1, 100)
    gamma = trial.suggest_loguniform("gamma", 0.01, 100)

    svr = SVR(kernel=kernel, C=C, gamma=gamma)
    scores = cross_val_score(svr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

    return scores.mean() * -1

Определение гиперпараметров с использованием библиотеки Optuna

In [16]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

best_params = study.best_params

[I 2024-12-16 19:22:45,444] A new study created in memory with name: no-name-ee4666f7-3fcb-4824-b297-54de083828c2
<ipython-input-13-3b1b257d3ae8>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform("gamma", 0.01, 100)
[I 2024-12-16 19:22:55,653] Trial 0 finished with value: 0.0024437447723003694 and parameters: {'kernel': 'poly', 'C': 69.9377097661063, 'gamma': 0.039726275380260835}. Best is trial 0 with value: 0.0024437447723003694.
<ipython-input-13-3b1b257d3ae8>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform("gamma", 0.01, 100)
[I 2024-12-16 19:23:00,189] Trial 1 finished with value: 0.00244256752601425

Обучение модели

In [17]:
final_model = SVR(**best_params)
final_model.fit(X_train, y_train)

SVR(C=7.6906534959280215, gamma=88.02270618485841, kernel='poly')

Прогноз

In [18]:
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

Метрики

In [19]:
r2_train = r2_score(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

r2_test = r2_score(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f'Метрики на тренировочном наборе:\n'
      f'r2: {r2_train:.4f}\n'
      f'mse: {mse_train:.4f}\n'
      f'mae: {mae_train:.4f}')

print(f'\nМетрики на тестовом наборе:\n'
      f'r2: {r2_test:.4f}\n'
      f'mse: {mse_test:.4f}\n'
      f'mae: {mae_test:.4f}')

Метрики на тренировочном наборе:
r2: 0.9977
mse: 0.0023
mae: 0.0397

Метрики на тестовом наборе:
r2: 0.9977
mse: 0.0023
mae: 0.0397
